In [213]:
from compChem.Hartree_Fock import Molecule
from scipy.linalg import eigh, sqrtm
import numpy as np
import psi4

In [3]:
def basischanger(c, f):
        """
        allows a change in basis
        
        input:
        c: transformation matrix
        f: operator to be transformed
        """
        return np.einsum("ij,jm,mn->in", c.T, f, c, optimize=True)

In [178]:
class ConstrainedMolecule:
    def __init__(self, geom_file):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
        #getting some old and some new matrices
        self.gamma_alpha = self.id.getDensityMatrix("alpha")
        self.gamma_beta = self.id.getDensityMatrix("beta")
        self.pMatrix = (self.gamma_alpha + self.gamma_beta)/2
        self.transform = eigh(self.pMatrix)[1]
        
    
    def fock_Alternator(self, spin):
        "alters a fock operator to follow CUHF"
        assert spin == "alpha" or spin == "beta", f"{spin}: no valid spin"
        f = self.id.displayFockMatrix(spin)
        f_trans = basischanger(self.transform, f)
        coulomb_a = np.einsum("nopq,pq->no", self.id.displayElectronRepulsion(), self.gamma_beta, optimize=True)
        exchange = np.einsum("npoq,pq->no", self.id.displayElectronRepulsion(), self.gamma_beta, optimize=True)
        closed_shell_type = 0.5*(self.id.displayHamiltonian() + 2*coulomb_a - exchange)
        closed_shell_trans = basischanger(self.transform, closed_shell_type)
        
        # amount of paired electrons
        cc = self.id.beta
        
        # amount of unpaored electrons
        oo = self.id.alpha - self.id.beta
        
        # amount of unoccupied orbitals
        
        # alter first block
        f_trans[:cc + oo, cc + oo:] = closed_shell_trans[:cc + oo, cc + oo:]
        
        # alter block in transposed position
        f_trans[cc + oo:, :cc + oo] = closed_shell_trans[cc + oo:, :cc + oo]
        
        
        return f_trans
        
    def iteratinator(self):
        self.id.setGuess(self.fock_Alternator("alpha"), "alpha")
        self.id.setGuess(self.fock_Alternator("beta"), "beta")
        return self.id.iterator(iteration=1000)
    
    
    def updatePMatrix(self):
        """updates pMatrix to current self.id densitymatrices"""
        self.pMatrix = (self.id.getDensityMatrix("alpha") + self.id.getDensityMatrix("beta"))

In [179]:
psi4.set_options({"BASIS": "cc-pvdz"})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-10)
h1.iteratinator()

/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":
/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert self.guessMatrix_a != "empty" and self.guessMatrix_b != "empty", "make a guess first"


iteration: 0, E_tot:  3.09969784, E_elek:  1.51216620, deltaE:  1.51216620, rmsD:  53.66265713
iteration: 1, E_tot: -1.45005526, E_elek: -3.03758689, deltaE: -4.54975309, rmsD:  2.49666388
iteration: 2, E_tot: -1.49832839, E_elek: -3.08586002, deltaE: -0.04827313, rmsD:  0.91285884
iteration: 3, E_tot: -1.50431270, E_elek: -3.09184434, deltaE: -0.00598432, rmsD:  0.42318556
iteration: 4, E_tot: -1.50562150, E_elek: -3.09315313, deltaE: -0.00130879, rmsD:  0.21237624
iteration: 5, E_tot: -1.50601873, E_elek: -3.09355037, deltaE: -0.00039724, rmsD:  0.11515600
iteration: 6, E_tot: -1.50616243, E_elek: -3.09369406, deltaE: -0.00014370, rmsD:  0.06760700
iteration: 7, E_tot: -1.50621974, E_elek: -3.09375137, deltaE: -0.00005731, rmsD:  0.04436653
iteration: 8, E_tot: -1.50624403, E_elek: -3.09377566, deltaE: -0.00002429, rmsD:  0.03202657
iteration: 9, E_tot: -1.50625485, E_elek: -3.09378648, deltaE: -0.00001082, rmsD:  0.02441250
iteration: 10, E_tot: -1.50625995, E_elek: -3.09379158, del

iteration: 104, E_tot: -1.50627429, E_elek: -3.09380593, deltaE: -0.00000000, rmsD:  0.00076108
iteration: 105, E_tot: -1.50627430, E_elek: -3.09380593, deltaE: -0.00000000, rmsD:  0.00073687
iteration: 106, E_tot: -1.50627430, E_elek: -3.09380593, deltaE: -0.00000000, rmsD:  0.00071344
iteration: 107, E_tot: -1.50627430, E_elek: -3.09380593, deltaE: -0.00000000, rmsD:  0.00069075
iteration: 108, E_tot: -1.50627430, E_elek: -3.09380593, deltaE: -0.00000000, rmsD:  0.00066878
iteration: 109, E_tot: -1.50627430, E_elek: -3.09380593, deltaE: -0.00000000, rmsD:  0.00064752
iteration: 110, E_tot: -1.50627430, E_elek: -3.09380594, deltaE: -0.00000000, rmsD:  0.00062693
iteration: 111, E_tot: -1.50627430, E_elek: -3.09380594, deltaE: -0.00000000, rmsD:  0.00060699
iteration: 112, E_tot: -1.50627431, E_elek: -3.09380594, deltaE: -0.00000000, rmsD:  0.00058769
iteration: 113, E_tot: -1.50627431, E_elek: -3.09380594, deltaE: -0.00000000, rmsD:  0.00056901
iteration: 114, E_tot: -1.50627431, E_el

iteration: 212, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00002326
iteration: 213, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00002252
iteration: 214, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00002181
iteration: 215, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00002112
iteration: 216, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00002045
iteration: 217, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00001980
iteration: 218, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00001917
iteration: 219, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00001856
iteration: 220, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00001797
iteration: 221, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00001740
iteration: 222, E_tot: -1.50627432, E_el

iteration: 316, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000081
iteration: 317, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000078
iteration: 318, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000076
iteration: 319, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000074
iteration: 320, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000071
iteration: 321, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000069
iteration: 322, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000067
iteration: 323, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000065
iteration: 324, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000063
iteration: 325, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000061
iteration: 326, E_tot: -1.50627432, E_el

iteration: 423, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000003
iteration: 424, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000002
iteration: 425, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 426, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000002
iteration: 427, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000002
iteration: 428, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000002
iteration: 429, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 430, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 431, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 432, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000002
iteration: 433, E_tot: -1.50627432, E_el

iteration: 531, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 532, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 533, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 534, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 535, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 536, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 537, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 538, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 539, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 540, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 541, E_tot: -1.50627432, E_el

-1.5062743202993865

In [83]:
h_test = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")


In [230]:
a = h_test.id.getDensityMatrix('alpha')
f_a = h_test.id.displayFockMatrix('alpha')

c = eigh(f_a)

s = h_test.id.overlap
p = h_test.pMatrix
s_eigh = eigh(s)
s_diag = np.diag(s_eigh[0])**(-1/2)
x = s_eigh[1].dot(s_diag).dot(s_eigh[1])
x

/usr/local/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in power
  if __name__ == '__main__':


array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       

In [158]:
f = (a + d)/2
g = np.einsum("pq,pq->pq", f, b, optimize=True)

In [174]:
h = g.sum(axis=1)
h_diag = np.diag(h)
c = eigh(h_diag)

In [173]:
f = c[1].T.dot(h_test.id.displayFockMatrix("alpha")).dot(c[1])
g = c[1].T.dot(h_test.id.displayFockMatrix("beta")).dot(c[1])

In [175]:
coulomb_a = np.einsum("nopq,pq->no", h_test.id.displayElectronRepulsion(), d, optimize=True)
exchange = np.einsum("npoq,pq->no", h_test.id.displayElectronRepulsion(), d, optimize=True)
closed_shell_type = 0.5*(h_test.id.displayHamiltonian() + 2*coulomb_a - exchange)
closed_shell_trans = basischanger(h_test.transform, closed_shell_type)

In [180]:
f[:3, 3:] = closed_shell_trans[:3, 3:]
        
g[3:, :3] = closed_shell_trans[3:, :3]

In [184]:
h_test.id.setGuess(f, "alpha")
h_test.id.setGuess(g, "beta")

h_test.id.iterator()

/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":
/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert self.guessMatrix_a != "empty" and self.guessMatrix_b != "empty", "make a guess first"


iteration: 0, E_tot:  4.02554290, E_elek:  2.43801126, deltaE:  2.43801126, rmsD:  51.50223492
iteration: 1, E_tot: -1.44222271, E_elek: -3.02975434, deltaE: -5.46776560, rmsD:  3.24356258
iteration: 2, E_tot: -1.49855362, E_elek: -3.08608526, deltaE: -0.05633092, rmsD:  0.76395553
iteration: 3, E_tot: -1.50372533, E_elek: -3.09125696, deltaE: -0.00517170, rmsD:  0.35008303
iteration: 4, E_tot: -1.50481998, E_elek: -3.09235161, deltaE: -0.00109466, rmsD:  0.22928500
iteration: 5, E_tot: -1.50518589, E_elek: -3.09271752, deltaE: -0.00036591, rmsD:  0.17220386
iteration: 6, E_tot: -1.50533619, E_elek: -3.09286782, deltaE: -0.00015030, rmsD:  0.14050362
iteration: 7, E_tot: -1.50540761, E_elek: -3.09293924, deltaE: -0.00007142, rmsD:  0.12034355
iteration: 8, E_tot: -1.50544769, E_elek: -3.09297932, deltaE: -0.00004008, rmsD:  0.10825182
iteration: 9, E_tot: -1.50547493, E_elek: -3.09300656, deltaE: -0.00002725, rmsD:  0.10154084
iteration: 10, E_tot: -1.50549692, E_elek: -3.09302855, del

iteration: 110, E_tot: -1.50627082, E_elek: -3.09380245, deltaE: -0.00000023, rmsD:  0.00901484
iteration: 111, E_tot: -1.50627104, E_elek: -3.09380267, deltaE: -0.00000022, rmsD:  0.00872676
iteration: 112, E_tot: -1.50627124, E_elek: -3.09380287, deltaE: -0.00000020, rmsD:  0.00844788
iteration: 113, E_tot: -1.50627143, E_elek: -3.09380307, deltaE: -0.00000019, rmsD:  0.00817790
iteration: 114, E_tot: -1.50627161, E_elek: -3.09380325, deltaE: -0.00000018, rmsD:  0.00791654
iteration: 115, E_tot: -1.50627178, E_elek: -3.09380342, deltaE: -0.00000017, rmsD:  0.00766354
iteration: 116, E_tot: -1.50627194, E_elek: -3.09380357, deltaE: -0.00000016, rmsD:  0.00741863
iteration: 117, E_tot: -1.50627209, E_elek: -3.09380372, deltaE: -0.00000015, rmsD:  0.00718155
iteration: 118, E_tot: -1.50627223, E_elek: -3.09380386, deltaE: -0.00000014, rmsD:  0.00695205
iteration: 119, E_tot: -1.50627236, E_elek: -3.09380399, deltaE: -0.00000013, rmsD:  0.00672990
iteration: 120, E_tot: -1.50627248, E_el

iteration: 214, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00031097
iteration: 215, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00030108
iteration: 216, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00029151
iteration: 217, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00028225
iteration: 218, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00027328
iteration: 219, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00026459
iteration: 220, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00025619
iteration: 221, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00024804
iteration: 222, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00024016
iteration: 223, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00023253
iteration: 224, E_tot: -1.50627432, E_el

iteration: 325, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000863
iteration: 326, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000836
iteration: 327, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000809
iteration: 328, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000784
iteration: 329, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000759
iteration: 330, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000735
iteration: 331, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000711
iteration: 332, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000689
iteration: 333, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000667
iteration: 334, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000646
iteration: 335, E_tot: -1.50627432, E_el

-1.5062743202993447

In [97]:
a = h_test.id.getDensityMatrix("alpha")
a_trans = z.T.dot(a).dot(z)

a_trans

array([[ 7.86993446e-23, -4.40492022e-29, -1.60209546e-24,
         1.10167228e-16,  6.31851038e-13,  6.80554947e-13,
        -2.47474297e-16, -6.62364160e-13,  5.36309038e-14,
         2.54238109e-14,  3.77365032e-13,  7.03392293e-13,
        -5.30833893e-14,  4.65973030e-12, -8.89193826e-12],
       [-4.40492022e-29,  2.76036507e-35,  8.96723218e-31,
        -6.16626961e-23, -2.50296198e-19, -5.15076464e-19,
         1.91198853e-22,  4.01685377e-19, -6.81987441e-20,
        -1.00010858e-20, -2.88968418e-19, -5.07425163e-19,
         2.10958303e-20, -1.66079758e-18,  5.69177369e-18],
       [-1.60209546e-24,  8.96723218e-31,  3.26141200e-26,
        -2.24269232e-18, -1.28624713e-14, -1.38544592e-14,
         5.03798893e-18,  1.34839228e-14, -1.09185579e-15,
        -5.17547528e-16, -7.68225020e-15, -1.43193185e-14,
         1.08060860e-15, -9.48568423e-14,  1.81016193e-13],
       [ 1.10167228e-16, -6.16626961e-23, -2.24269232e-18,
         1.54217524e-10,  8.84480564e-07,  9.52694805

In [45]:
a_trans.dot(a_trans)

array([[ 1.31509744e-32, -1.14508632e-33,  3.39607801e-48,
        -1.61963005e-49,  2.05562799e-49, -9.73656549e-38,
         6.24182271e-35, -1.09179883e-33, -6.00573899e-34,
         8.81299316e-33, -6.04751451e-33, -1.33620208e-33,
         6.66989478e-33,  5.38281907e-17, -3.80353931e-17],
       [ 2.55329854e-33,  1.82436891e-33,  1.34535402e-47,
         1.56847134e-49,  1.23531439e-48,  4.82548374e-37,
         1.88662530e-34, -4.78299584e-34,  1.31304341e-33,
         5.06955630e-34, -8.48027587e-33,  3.23997069e-34,
         1.42303235e-33,  5.40898234e-18,  5.72211595e-17],
       [ 1.46042575e-47,  1.16065337e-47,  8.83747223e-62,
         1.07094341e-63,  8.17295333e-63,  3.26345890e-51,
         1.35252332e-48, -2.70869588e-48,  9.32448840e-48,
         2.66324006e-48, -5.41008378e-47,  2.35858984e-48,
         8.41908870e-48,  3.15960266e-32,  3.71914730e-31],
       [ 1.79514095e-50,  1.35123144e-49,  8.48917413e-64,
         1.24083986e-65,  7.91306457e-65,  3.28005275

In [32]:
c = eigh(a)[0]

In [33]:
b = h_test.id.getDensityMatrix("beta")
b.trace()

0.4718682903812628

In [34]:
d = eigh(b)[0]

In [35]:
e = np.diag(c) + np.diag(d)
e/2

array([[-1.01658530e-17,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.29458077e-18,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.57922657e-19,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.53354521e-19,  0.00000000e+00,  0.00000000